In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [ ]:
df = pd.read_csv('main_dataset.csv').fillna(method="ffill")
df=df[:300000]
df.head()
df.isnull().sum()
df = df.fillna(method='ffill')

df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()
df1=df.groupby('Tag').size().reset_index(name='counts')

class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df).sentences

sentences = []
labels = []
for s in getter:
    qw = []
    qw2 = []
    for j in s:
        qw.append(j[0])
        qw2.append(j[2])

    sentences.append(qw)
    labels.append(qw2)
print(df.head())

    Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1  Sentence: 1             of   IN   O
2  Sentence: 1  demonstrators  NNS   O
3  Sentence: 1           have  VBP   O
4  Sentence: 1        marched  VBN   O


In [ ]:
print(len(sentences))
print(len(labels))

13744
13744


In [ ]:
list1 = []
f = open("shuffled.csv", "r")
curr = []
for x in f:
    if "Sentence #" in x:
        if curr != [] :
            list1.append(curr)
        curr = []
    else :
        # x = x[0:len(x)-1]
        y = x.strip('\n').split(',')
        if y[1]!='':
            curr.append(y)
for i in list1:
    s = []
    l = []
    for j in i:
        s.append(j[0])
        l.append(j[1])
    sentences.append(s)
    labels.append(l)

In [ ]:
var2=[]
for i in labels:
  var2 +=i
tag_values = list(set(var2))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
import torch

In [ ]:
pip install transformers==2.6.0

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'1.6.0+cu101'

In [ ]:
MAX_LEN = 75
bs = 32

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")
print(tags)

[[0 0 0 ... 4 4 4]
 [0 0 0 ... 4 4 4]
 [0 0 0 ... 4 4 4]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 4 4 4]
 [0 0 0 ... 0 0 0]]


In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__

'2.6.0'

In [ ]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

In [ ]:
model.cuda();

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 15
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
pip install seqeval

In [ ]:
from seqeval.metrics import f1_score, accuracy_score, precision_score, recall_score

In [ ]:
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print("Validation Precision: {}".format(precision_score(pred_tags, valid_tags)))
    print("Validation Recall: {}".format(recall_score(pred_tags, valid_tags)))
    print()

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Average train loss: 0.056820828342274234
Validation loss: 0.07821485141499175
Validation Accuracy: 0.9760091967886623
Validation F1-Score: 0.8266627856207248


Epoch:   7%|▋         | 1/15 [03:32<49:37, 212.70s/it]

Validation Precision: 0.8222350897510133
Validation Recall: 0.8311384255194615

Average train loss: 0.04117672618547952
Validation loss: 0.08639836300992304
Validation Accuracy: 0.9750103652331236
Validation F1-Score: 0.8283333333333334
Validation Precision: 0.863346844238564


Epoch:  13%|█▎        | 2/15 [07:05<46:03, 212.60s/it]

...

Validation Precision: 0.8401852924145917
Validation Recall: 0.8305666857469949

Average train loss: 0.003909793757238599
Validation loss: 0.17215220029983255
Validation Accuracy: 0.9758207380045983
Validation F1-Score: 0.8315319210260845


Epoch:  93%|█████████▎| 14/15 [49:28<03:31, 211.67s/it]

Validation Precision: 0.8352634626519977
Validation Recall: 0.8278335724533716

Average train loss: 0.003453795028325061
Validation loss: 0.1796703517988876
Validation Accuracy: 0.9760091967886623


Epoch: 100%|██████████| 15/15 [52:59<00:00, 211.99s/it]

Validation F1-Score: 0.8347202295552366
Validation Precision: 0.8422119281991893
Validation Recall: 0.8273606370875995



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls 

gdrive	main_dataset.csv  sample_data  shuffled.csv


In [ ]:
%cd 'gdrive'

/content/gdrive


In [ ]:
%cd 'My Drive'

/content/gdrive/My Drive


In [ ]:
model_save_name = 'BERT_LOC.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
test_sentence = """To BSES Dear Sir /Madam please don't allow any type of electricity Connection to this property address B 69 GALI NO 5 JYOTI COLONY LONI ROAD SHAHDARA DELHI 110032 because this property pending court case ownership dispute pending in high court delhi some people are Fraudster
"""

In [ ]:
tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence]).cuda()

In [ ]:
with torch.no_grad():
    output = model(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

In [ ]:
# join bpe split tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)

In [ ]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

O	[CLS]
O	To
O	BSES
O	Dear
O	Sir
O	/
O	Madam
O	please
O	don
O	'
O	t
O	allow
O	any
O	type
O	of
O	electricity
O	Connection
O	to
O	this
O	property
O	address
B-LOC	B
I-LOC	69
B-LOC	GALI
I-LOC	NO
I-LOC	5
B-LOC	JYOTI
I-LOC	COLONY
B-LOC	LONI
I-LOC	ROAD
B-LOC	SHAHDARA
B-LOC	DELHI
I-LOC	110032
O	because
O	this
O	property
O	pending
O	court
O	case
O	ownership
O	dispute
O	pending
O	in
O	high
O	court
B-LOC	delhi
O	some
O	people
O	are
O	Fraudster
O	[SEP]


In [ ]:
model_save_name = 'spanbert_new_new.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model.load_state_dict(torch.load(path))

FileNotFoundError: ignored

In [ ]:
test_sentence = """
Request you to take action and resolve this issue as early as possible
Residents have already escalated but no satisfactory response received that is why I am drawing your attention towards this matter 
@ArvindKejriwal

@raghav_chadha
 
@DelhiJalBoard
 
 B-block  318 Gali No 5 Hari Nagar. 
"""

In [ ]:
tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence]).cuda()

In [ ]:
with torch.no_grad():
    output = model(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

In [ ]:
# join bpe split tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)

In [ ]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

O	[CLS]
O	Request
O	you
O	to
O	take
O	action
O	and
O	resolve
O	this
O	issue
O	as
O	early
O	as
O	possible
O	Residents
O	have
O	already
O	escalated
O	but
O	no
O	satisfactory
O	response
O	received
O	that
O	is
O	why
O	I
O	am
O	drawing
O	your
O	attention
O	towards
O	this
O	matter
O	@
O	ArvindKejriwal
O	@
O	raghav
O	_
O	chadha
O	@
O	DelhiJalBoard
B-LOC	B
I-LOC	-
B-LOC	block
I-LOC	318
B-LOC	Gali
I-LOC	No
I-LOC	5
B-LOC	Hari
I-LOC	Nagar
O	.
O	[SEP]


In [ ]:
test_sentence = """
Check distance from Gali No. 5, Dwarka to Sabzi Mandi Old, New Delhi, Delhi- 110036.
"""

In [ ]:
tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence]).cuda()

NameError: ignored

In [ ]:
with torch.no_grad():
    output = model(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

In [ ]:
# join bpe split tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)

In [ ]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

O	[CLS]
O	Check
O	distance
O	from
B-LOC	Gali
I-LOC	No
I-LOC	.
I-LOC	5
O	,
B-LOC	Dwarka
O	to
B-LOC	Sabzi
I-LOC	Mandi
I-LOC	Old
O	,
B-LOC	New
I-LOC	Delhi
O	,
B-LOC	Delhi
I-LOC	-
I-LOC	110036
O	.
I-LOC	[SEP]
